# DA- VAE, Diffusion (MDD_HRV_JA) - for bipolar main_dx group

* Using Keras/Tensorflow

In [ ]:
import os
import time
# import shap ## for XAI
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
# import pingouin as pg
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [ ]:
from numpy import dot
from numpy.linalg import norm

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.layers import Dense , Activation, Dropout, BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.callbacks import Callback

from scipy.special import rel_entr
from scipy.stats import gaussian_kde
from scipy.spatial.distance import jensenshannon

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, train_test_split, ParameterGrid
from sklearn import decomposition, metrics
from sklearn.cluster import KMeans
from sklearn.metrics import cohen_kappa_score,f1_score, confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.manifold import TSNE

from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Lambda, Conv1D, Flatten, Reshape, UpSampling1D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import mse, MeanSquaredError

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import tensorflow as tf 
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

***

---

## Original data check

In [ ]:
data_ori = pd.read_csv('E:/RESEARCH/Datasets/HRV/JA/HRV_bipolar.csv')

In [ ]:
data_ori.shape

In [ ]:
data_ori.columns

In [ ]:
data_ori["main_dx"].value_counts()

In [ ]:
# # 결측값이 하나라도 있는 행의 개수 확인
# num_missing_rows = data_ori.isna().any(axis=1).sum()
# print(f"The number of rows that contains at least one missing value: {num_missing_rows}")

### Check and split features

In [ ]:
hrv_feature_list = ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln']
hrv_feature_list_core = ['sdnn', 'tp','vlf','lfnorm','hfnorm','rmssd','apen','srd','tsrd']

In [ ]:
##### for new "depression" classification
hrv_all = data_ori.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln','main_dx']]
hrv_core = data_ori.loc[:, ['sdnn', 'tp','vlf','lfnorm','hfnorm','rmssd','apen','srd','rmssd','main_dx']]
hrv_multi = data_ori.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln',
                             'auto_activity','auto_balance','stress_resist','stress_index','tired','avg_hr','heart_stable','abnormal_hr','main_dx']]

### Data selection (on specific group)

In [ ]:
# data_disorder = hrv_all
# data_disorder = hrv_core
data_disorder = hrv_multi

In [ ]:
data_bpi = data_disorder[data_disorder['main_dx']=="BP_I"]
data_bpii = data_disorder[data_disorder['main_dx']=="BP_II"]

In [ ]:
data_disorder.columns

In [ ]:
data_disorder['main_dx'].value_counts()

In [ ]:
data_vis = data_disorder.copy()
data_vis = data_vis.drop(['main_dx'], axis=1)

In [ ]:
features = [col for col in list(data_vis.columns)]

In [ ]:
def kdeplot_features(data, feature, title):
    
    df0 = data[data['main_dx']=="BP_I"]
    df1 = data[data['main_dx']=="BP_II"]

    df0 = df0.drop(['main_dx'], axis=1)
    df1 = df1.drop(['main_dx'], axis=1)
    
    df0_values = df0[feature].to_numpy()
    df1_values = df1[feature].to_numpy()
     
    plt.figure(figsize = (6, 2.5))
    
    sns.kdeplot(df0_values, color = 'b')
    sns.kdeplot(df1_values, color = 'r')
    
    plt.title(title, fontsize=15)
    plt.legend()
    plt.show();
    
    # del values_train , values_test
    # gc.collect()

In [ ]:
for var in features:
    kdeplot_features(data_disorder, feature=var, title = var + "distribution")

--------------

----------

## 1D VAE model

In [ ]:
data_disorder["main_dx"].value_counts()

> Checking hrv_bdi dataset, it requires balance for lacking severity data group \
> underneath, class is labeled 0(normal), 1(mild), 2(moderate), 3(severe) \
> BDI Classification: ratio should be 1.99 * (normal), 3.39 * (mild), 1.85 * (moderate), 1 * (severe) \
> HAMD Classification: ratio shoudl be 6.78 * (normal), 1 * (mild), 1.78 * (moderate), 6.74 * (severe)

In [ ]:
# data = data_disorder
# data = data_bpi
data = data_bpii

In [ ]:
class Args:
    # arugments
    epochs=100
    bs=32
    lr=0.001
    momentum=0.9
    num_classes= 2
    seed=710674

args = Args()

In [ ]:
data.columns

In [ ]:
### Using MDD-HRV -- depression class
x = data.drop(['main_dx'], axis=1)
y = data.loc[:,'main_dx']

In [ ]:
x = x.fillna(x.mean())

In [ ]:
### MDD - HRV ####### FULL GROUP CLASSIFICATION ##########
label = y
label = label.replace({'BP_I':0})
label = label.replace({'BP_II': 1})

data_y = to_categorical(label, 2) ## into the format of one-hot encoding

In [ ]:
data_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)
data_x[:] = scaler.fit_transform(data_x[:])
data_x = data_x.round(decimals=5)

In [ ]:
print("The size of x dataset is:", data_x.shape)
print("The size of y dataset is:", data_y.shape)

In [ ]:
# data_x.head()
data_x.shape

### encoder networks

In [ ]:
input_dim = data_x.shape[1]
latent_dim = 2

inputs = Input(shape=(input_dim,))
h = Dense(32, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [ ]:
# ## deeper model
# input_dim = data_x.shape[1]
# latent_dim = args.latent_dim

# inputs = Input(shape=(input_dim,))
# h = Dense(128, activation='relu')(inputs)
# h = Dense(64, activation='relu')(h)
# h = Dense(32, activation='relu')(h)
# z_mean = Dense(latent_dim)(h)
# z_log_var = Dense(latent_dim)(h)

In [ ]:
## latent space sampling
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

### decoder networks

In [ ]:
decoder_h = Dense(32, activation='relu')
decoder_mean = Dense(input_dim, activation='sigmoid')

h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

In [ ]:
# ## deeper model
# decoder_h1 = Dense(32, activation='relu')
# decoder_h2 = Dense(64, activation='relu')
# decoder_h3 = Dense(128, activation='relu')
# decoder_mean = Dense(input_dim, activation='sigmoid')

# h_decoded = decoder_h1(z)
# h_decoded = decoder_h2(h_decoded)
# h_decoded = decoder_h3(h_decoded)
# x_decoded_mean = decoder_mean(h_decoded)

### model define

In [ ]:
vae = Model(inputs, x_decoded_mean)
# vae.summary()

### loss function

In [ ]:
reconstruction_loss = MeanSquaredError()(inputs, x_decoded_mean)

kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var + K.epsilon()), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

In [ ]:
vae.compile(optimizer=Adam(learning_rate=0.001))

### model training

In [ ]:
vae.fit(data_x, epochs=args.epochs, batch_size=args.bs, validation_split=0.2, verbose=2)

### extracting encoder, decoder

In [ ]:
## encoder model extraction
encoder = Model(inputs, z_mean)

In [ ]:
## decoder model extraction
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
decoder = Model(decoder_input, _x_decoded_mean)

In [ ]:
# #### deeper model
# decoder_input = Input(shape=(latent_dim,))
# _h_decoded = decoder_h1(decoder_input)
# _h_decoded = decoder_h2(_h_decoded)
# _h_decoded = decoder_h3(_h_decoded)
# _x_decoded_mean = decoder_mean(_h_decoded)
# decoder = Model(decoder_input, _x_decoded_mean)

### Latent space visualization

In [ ]:
# ## encode the input dataset into latent space
# encoded_data = encoder.predict(data_x)

# ## latent space transform with t-SNE function
# tsne = TSNE(n_components=2, random_state=710674)
# encoded_data_tsne = tsne.fit_transform(encoded_data)

# ## visualize
# plt.figure(figsize=(8, 6))
# plt.scatter(encoded_data_tsne[:, 0], encoded_data_tsne[:, 1], c=label, cmap='viridis')
# plt.colorbar()
# plt.xlabel("t-SNE component 1")
# plt.ylabel("t-SNE component 2")
# plt.title("t-SNE visualization of the latent space")
# plt.show()

### Synthetic data generation

> Our dataset contains 33 BP_I, 204 BP_II \
> set the target number as 300 \
> So, augment should be 267 BP_I, 96 BP_II

In [ ]:
### sampling randomly from latent space
# n_samples = 267  # for normal gen
n_samples = 96  # for severe gen

z_samples = np.random.normal(size=(n_samples, latent_dim))

### 디코더를 통해 synthetic data 생성
# synthetic_data_bpi = decoder.predict(z_samples)
synthetic_data_bpii = decoder.predict(z_samples)

In [ ]:
# synthetic_data_nor.shape
# synthetic_data_mil.shape
# synthetic_data_mod.shape
# synthetic_data_sev.shape

In [ ]:
# ####################
# gen_bpi = synthetic_data_bpi.copy()
# gen_bpi = pd.DataFrame(gen_bpi, columns=data_x.columns)

#####################
gen_bpii = synthetic_data_bpii.copy()
gen_bpii = pd.DataFrame(gen_bpii, columns=data_x.columns)

In [ ]:
gen_bpii.shape

----------

-----------

## Latent Diffusion Model

### encoder, decoder networks

In [ ]:
# Sampling function for the latent space
def sampling(repara):
    z_mean, z_log_var = repara
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define encoder
def build_encoder(input_shape, latent_dim):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 3, activation='relu', padding='same')(inputs)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)
    return Model(inputs, [z_mean, z_log_var], name='encoder')

# Define decoder
def build_decoder(output_shape, latent_dim):
    latent_inputs = Input(shape=(latent_dim,))
    x = Dense(128, activation='relu')(latent_inputs)
    x = Dense(np.prod(output_shape), activation='relu')(x)
    x = Reshape(output_shape)(x)
    outputs = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
    return Model(latent_inputs, outputs, name='decoder')

In [ ]:
input_shape = (x_train.shape[1], 1)  # Example input shape
latent_dim = 16

In [ ]:
## Define the VAE
encoder = build_encoder(input_shape, latent_dim)
decoder = build_decoder(input_shape, latent_dim)

inputs = Input(shape=input_shape)
z_mean, z_log_var = encoder(inputs)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
outputs = decoder(z)

vae = Model(inputs, outputs, name='vae')

# Define the VAE loss
reconstruction_loss = MeanSquaredError()(inputs, outputs)
kl_loss = -0.5 * tf.reduce_mean(
    z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae_loss = reconstruction_loss + kl_loss
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

vae.summary()

In [ ]:
encoder.summary()

In [ ]:
decoder.summary()

### model training

In [ ]:
# Train the VAE
vae.fit(x_train, epochs=args.epochs, batch_size=args.bs, verbose=2)

### practice on new dataset

In [ ]:
vae_prediction = vae.predict(x_train[:])

In [ ]:
vae_prediction.shape

In [ ]:
vae_prediction_re = vae_prediction.reshape(1729,356)

In [ ]:
vae_prediction_re.shape

---------

-----------

## DNN classification/prediction

### using original dataset only

In [ ]:
data = data_disorder

In [ ]:
data.columns

In [ ]:
data['main_dx'].value_counts()

In [ ]:
### Using MDD,BP dataset
x = data.drop(['main_dx'], axis=1)
y = data.loc[:,["main_dx"]]

In [ ]:
x = x.fillna(x.mean())

In [ ]:
### MDD - HRV ####### FULL GROUP CLASSIFICATION ##########

label = y
label = label.replace({'BP_I':0})
label = label.replace({'BP_II': 1})

data_y = to_categorical(label, 2) ## into the format of one-hot encoding

In [ ]:
data_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)
data_x[:] = scaler.fit_transform(data_x[:])
data_x = data_x.round(decimals=5)

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(data_x, data_y, test_size = 0.2, random_state = 710674)

In [ ]:
x_train, x_vali, y_train, y_vali = train_test_split(x_trainset, y_trainset, test_size = 0.1, random_state = 710674)

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

***

### using original + generated dataset

> we generate synthesized dataset using VAE from above. \
> adopted original datset: data_mdd, data_bpi, data_bpii \
> synthesized into: gen_mdd, gen_bpi, gen_bpii \
> gen_control is not generated since "control group" has biggest number.

* Set target information into newly generated dataset

In [ ]:
gen_bpi['main_dx'] = "BP_I"
gen_bpii['main_dx'] = "BP_II"

In [ ]:
data_bpi.columns

* Original dataframe concat

In [ ]:
ori_df_list = [data_bpi, data_bpii]
ori_df_concat = pd.concat(ori_df_list, ignore_index=True)

In [ ]:
ori_df_concat.shape

In [ ]:
ori_df_concat['main_dx'].value_counts()

In [ ]:
gen_df_list = [gen_bpi, gen_bpii]
gen_df_concat = pd.concat(gen_df_list, ignore_index=True)

In [ ]:
gen_df_concat['main_dx'].value_counts()

In [ ]:
### Using MDD,BP dataset
ori_x = ori_df_concat.drop(['main_dx'], axis=1)
ori_y = ori_df_concat.loc[:,["main_dx"]]

gen_x = gen_df_concat.drop(['main_dx'], axis=1)
gen_y = gen_df_concat.loc[:,["main_dx"]]

In [ ]:
ori_x = ori_x.fillna(ori_x.mean())

In [ ]:
label_ = ori_y
label_ = label_.replace({'BP_I':0})
label_ = label_.replace({'BP_II': 1})

y_ori = to_categorical(label_, 2) ## into the format of one-hot encoding

In [ ]:
label = gen_y
label = label.replace({'BP_I':0})
label = label.replace({'BP_II': 1})

y_gen = to_categorical(label, 2) ## into the format of one-hot encoding

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(ori_x, y_ori, test_size = 0.3, random_state = 710674)

In [ ]:
x_train_concat = pd.concat([x_trainset, gen_x], ignore_index=True)
y_train_concat = np.concatenate([y_trainset, y_gen])

In [ ]:
x_train, x_vali, y_train, y_vali = train_test_split(x_train_concat, y_train_concat, test_size = 0.2, random_state = 710674)

In [ ]:
ori_x[ori_x.isna().any(axis=1)].shape

In [ ]:
# # fig, ax = plt.subplots()
# plt.figure(figsize = (10, 5))
# plt.xlabel('Generated MDD BP feature variables',fontsize=10)
# plt.ylabel('Feature value',fontsize=10)
# plt.boxplot(ori_x)
# plt.show()

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

***

### model training + test

In [ ]:
class Args:
    # arugments
    epochs=150
    bs=32
    lr=0.001
    momentum=0.9
    num_classes= 2
    verbose='store_true'
    seed=710674

args = Args()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
fold_num = 1
split_num = 5
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-6, momentum = args.momentum)
kfold = KFold(n_splits = split_num, shuffle = True)

In [ ]:
data['main_dx'].value_counts()

In [ ]:
### class weight
# class_weight = {0:6.5, 1:1}
class_weight = {0:1, 1:1}

In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
x_train.shape

In [ ]:
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Dense(32, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(args.num_classes, activation = 'softmax'))
    
    ## model compile
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    print('----------------------------------------')
    print(f'Training or fold {fold_num} ... ')
    
    ## fit data to model
    history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    ## generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test])
    print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    ## increasing fold number
    fold_num = fold_num + 1
    
    
    
## Summarizing the results
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'>>> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("=============================================")

***

***

## Using VAE/AE as a dimensionality reduction

In [ ]:
data = data_no_s8

In [ ]:
# data_no_s8.shape

In [ ]:
# data_no_s8[data_no_s8.isna().any(axis=1)].shape

In [ ]:
### Using MDD,BP dataset
data_x = data.drop(['subject_info', 'birth_info', 'target_info', 'sex_info', 'age_info', 'family_hx_info'], axis=1)
data_y = data.loc[:,["target_info"]]

In [ ]:
data_x = data_x.fillna(data_x.mean())

In [ ]:
### MDD, BP ####### FULL GROUP CLASSIFICATION ##########
x = data_x

label = data_y
label = label.replace({'control':0})
label = label.replace({'mdd': 1})
label = label.replace({'bpi': 2})
label = label.replace({'bpii': 3})

y = to_categorical(label, 4) ## into the format of one-hot encoding

In [ ]:
class Args:
    # arugments
    epochs=200
    bs=32
    lr=0.001
    momentum=0.9
    encoding_dim = 32
    num_classes= 4
    verbose='store_true'
    seed=710674

args = Args()

### Autoencoder for dim reduction

In [ ]:
# input dataset layer
input_layer = Input(shape=(x.shape[1],))

# encoder layers
encoder = Dense(256, activation='relu')(input_layer)
encoder = Dense(128, activation='relu')(encoder)
encoder_out = Dense(args.encoding_dim, activation='relu')(encoder)

# decoder layers
decoder = Dense(128, activation='relu')(encoder_out)
decoder = Dense(256, activation='relu')(decoder)
decoder_out = Dense(x.shape[1], activation='sigmoid')(decoder)

In [ ]:
# AE model
autoencoder = Model(inputs=input_layer, outputs=decoder_out)

# Encoder model (convert input dataset into latent space)
encoder_model = Model(inputs=input_layer, outputs=encoder_out)

# Decoder model (recover latent space/vector into original dataset format)
encoded_input = Input(shape=(args.encoding_dim,))
decoder_layer = autoencoder.layers[-3](encoded_input)
decoder_layer = autoencoder.layers[-2](decoder_layer)
decoder_out = autoencoder.layers[-1](decoder_layer)
decoder_model = Model(inputs=encoded_input, outputs=decoder_layer)

In [ ]:
# Model compile
autoencoder.compile(optimizer='adam', loss='mse')

# Checking each model
autoencoder.summary()
# encoder_model.summary()
# decoder_model.summary()

In [ ]:
## Model training
autoencoder.fit(x, x, epochs = args.epochs, batch_size = args.bs, shuffle=True, validation_split=0.2, verbose=2)

In [ ]:
encoded_data = encoder_model.predict(x)
decoded_data = decoder_model.predict(encoded_data)

In [ ]:
encoded_data.shape

In [ ]:
decoded_data.shape

In [ ]:
y.shape

In [ ]:
class Args:
    # arugments
    epochs=200
    bs=32
    lr=0.001
    momentum=0.9
    num_classes= 4
    verbose='store_true'
    seed=710674

args = Args()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(decoded_data, y, test_size = 0.1, random_state = 710674)
x_train, x_vali, y_train, y_vali = train_test_split(x_trainset, y_trainset, test_size = 0.2, random_state = 710674)

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

In [ ]:
fold_num = 1
split_num = 5
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-6, momentum = args.momentum)
kfold = KFold(n_splits = split_num, shuffle = True)
# kfold = StratifiedKFold(n_splits = split_num, shuffle = True)

In [ ]:
############## FOR FOUR-GROUP CLASSIFICATION ###############
class_weight = {1:1.29, 2:3.04, 3: 1.13, 0:1}

In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Dense(128, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5)) #drop out
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(args.num_classes, activation = 'softmax'))
    
    ## model compile
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    print('----------------------------------------')
    print(f'Training or fold {fold_num} ... ')
    
    ## fit data to model
    history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    ## generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test])
    print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    ## increasing fold number
    fold_num = fold_num + 1
    
    
    
## Summarizing the results
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'>>> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("=============================================")